In [5]:
import numpy as np
import pandas as pd
import xarray as xr

In [6]:
path = r'/home/sachin/Documents/NIPR/Research/Data/REPPU/fbig5min.dat' #Server
#path = r'/home/sachin/Documents/NIPR/Research/Data/REPPU/fbig5minss.dat' #Server

#read the REPPU data
with open (path) as f:
    rectype = np.dtype(np.float32)
    reppu_data = np.fromfile(f, rectype) #size = 109,900,800

In [7]:
#Read MHD dates and expand-out the date ranges
mhd_data = pd.read_csv('mhd_dates.csv')
expanded_dt = pd.concat([pd.Series(pd.date_range(start, end)) 
        for start, end in zip(mhd_data['start'], mhd_data['end'])])

In [8]:
# Reshape the array to have dimensions (days, time (5-min intervals), lat, lon)
#288 will become 1440 at 1-min resolution in SMRAI v3.0
data_reshaped = reppu_data.reshape(len(expanded_dt), 288, 30, 80) 

# Define coordinates
time = np.arange(288)
lat = np.linspace(53.1, 89.7, 30) #30 intervals between 53.1° to 89.7°
lon = np.linspace(1.6, 357.6, 80) #80 intervals between 1.6° to 357.6°

# Create 'dt' variable combining dates and five-minute intervals
dt = []
for day in expanded_dt:
    for t in time:
        dt.append(day + pd.Timedelta(minutes=t*5))
dt = np.array(dt) #convert from list to numpy array

# Create xarray Dataset
ds = xr.Dataset({'current': (['dt', 'lat', 'lon'], data_reshaped.reshape(-1, 30, 80))},
                coords={'dt': dt, 'lat': lat, 'lon': lon})

# Add current and units
ds['current'].attrs['units'] = 'mA/m^2'
ds['current'] = ds['current'] * 1e6 #convert from A to uA
ds

<xarray.Dataset> Size: 440MB
Dimensions:  (dt: 45792, lat: 30, lon: 80)
Coordinates:
  * dt       (dt) datetime64[ns] 366kB 2021-05-10 ... 2022-07-31T23:55:00
  * lat      (lat) float64 240B 53.1 54.36 55.62 56.89 ... 87.18 88.44 89.7
  * lon      (lon) float64 640B 1.6 6.106 10.61 15.12 ... 348.6 353.1 357.6
Data variables:
    current  (dt, lat, lon) float32 440MB 0.0 0.0 0.0 ... 0.1796 0.1782 0.1768

In [15]:
import os

# Define the folder path
folder_path = '/run/media/sachin/0fa21ddb-f70c-4238-9cf4-705e0360f1c1/NICT DUMP/idata2020/idata202008/'

# List all files in the folder
files = os.listdir(folder_path)

# Filter for files that don't have an extension or check for possible .dat files
for file in files:
    file_path = os.path.join(folder_path, file)
    
    # Check if it's a file (not a directory)
    if os.path.isfile(file_path):
        try:
            # Open the file in binary mode ('rb')
            with open(file_path, 'rb') as f:
                content = f.read(100)  # Read the first 100 bytes
                print(f"Opened {file}:")
                print(content)  # Print raw binary content
        except Exception as e:
            print(f"Failed to open {file}: {e}")

    break  # Remove this if you want to process all files in the folder


Opened idata202008010000:
b'\x08\x00\x00\x00\x07\x04\x00\x00!\x88#E\x08\x00\x00\x00tT\x04\x00z\xd9\xd0\xb8z\xd9\xd0\xb8}\xd9\xd0\xb8~\xd9\xd0\xb8\x83\xd9\xd0\xb8\x89\xd9\xd0\xb8\x8e\xd9\xd0\xb8\x95\xd9\xd0\xb8\x9a\xd9\xd0\xb8\xa4\xd9\xd0\xb8\xac\xd9\xd0\xb8\xb4\xd9\xd0\xb8\xbc\xd9\xd0\xb8\xc6\xd9\xd0\xb8\xcf\xd9\xd0\xb8\xdb\xd9\xd0\xb8\xe4\xd9\xd0\xb8\xf0\xd9\xd0\xb8\xfa\xd9\xd0\xb8\x06\xda\xd0\xb8'


In [22]:
unpacked_data[0]

8